# N-Gram Language Models
In this exercise, we will use n-gram language models to predict the probability of text, and generate it.

In [1]:
from nltk.corpus import gutenberg

First, we load Jane Austen's Emma from NLTK's gutenberg corpus that we also used in a previous exercise. Tokenize and lowercase this text such that we have a list of words.

In [2]:
raw_text = gutenberg.raw('austen-emma.txt')
words = [w.lower() for w in raw_text.split()]
print(len(words))

158167


Write an n-gram language model class that takes the word list and a parameter `n` as inputs, where `n` is a positive integer larger than 1 that determines the `n` of the n-gram LM. The LM should build a dictionary of n-gram counts from the word list.

In [3]:
from collections import defaultdict

class NGramLanguageModel:
    
    def __init__(self, words, n=2):
        assert n > 1, "n needs to be a positive integer > 1"
        assert n <= len(words), "n can't be larger than the number of words"
        self.n = n
        self.counts = defaultdict(int)
        for i in range(len(words) - self.n + 1):
            ngram = tuple(words[i:i+self.n])
            self.counts[ngram] += 1
            self.counts[ngram[:-1]] += 1
        self.counts[ngram[1:]] += 1  # last n-gram in the input

Now we "train" the n-gram LM by building the n-gram counts of the Emma novel. Use a low `n` (i.e. 2 or 3).

In [4]:
ngrams = NGramLanguageModel(words, n=3)

Let's add a method `log_probability` to the n-gram LM class that computes the probability of an input string. Since multiplying many probabilities (<= 1) results in very small numbers that can underflow, we sum the log probabilities instead.

In [5]:
import math

def log_probability(self, input_string):
    log_prob = []
    words = [w.lower() for w in input_string.split()]
    for i in range(len(words) - self.n + 1):
        ngram = tuple(words[i:i+self.n])
        if ngram in self.counts:
            prob = self.counts[ngram] / self.counts[ngram[:-1]]
            log_prob.append(math.log(prob))
    return sum(log_prob)

NGramLanguageModel.log_probability = log_probability

In [6]:
print(ngrams.log_probability('There is a house in New Orleans.'))
print(ngrams.log_probability('There is a house in New Orleans. '
                             'And then there are also many other houses in many other cities.'))

-2.159484249353372
-5.049856007249537


Shorter texts will have higher log probability than longer texts, so we need to normalize it by the number of words in the input string.

In [7]:
import numpy as np

def log_probability(self, input_string):
    log_prob = []
    words = [w.lower() for w in input_string.split()]
    for i in range(len(words) - self.n + 1):
        ngram = tuple(words[i:i+self.n])
        if ngram in self.counts:
            prob = self.counts[ngram] / self.counts[ngram[:-1]]
            log_prob.append(math.log(prob))
    return np.mean(log_prob)

NGramLanguageModel.log_probability = log_probability

Lets predict the probabilities of two novels under our trained model: Jane Austen's *Sense and Sensibility* (`austen-sense.txt`) and Shakespeare's *Hamlet* (`shakespeare-hamlet.txt`).
- What do you expect will happen?
- What do you observe?

In [8]:
austen_sense = gutenberg.raw('austen-sense.txt')
shakespeare_hamlet = gutenberg.raw('shakespeare-hamlet.txt')
print(ngrams.log_probability('There is a house in New Orleans.'))
print(ngrams.log_probability(austen_sense))
print(ngrams.log_probability(shakespeare_hamlet))

-2.159484249353372
-2.142628255277218
-2.2824723388271106


How many n-grams are known in each input?

In [9]:
def known_ngrams(self, input_string):
    known = 0
    words = [w.lower() for w in input_string.split()]
    for i in range(len(words) - self.n + 1):
        ngram = tuple(words[i:i+self.n])
        if ngram in self.counts:
            known += 1
    return known, len(words) - self.n + 1

NGramLanguageModel.known_ngrams = known_ngrams
known, total = ngrams.known_ngrams(austen_sense)
print(f'Known n-grams in Sense and Sensibility: {known/total:.2%} ({known}/{total})')
known, total = ngrams.known_ngrams(shakespeare_hamlet)
print(f'Known n-grams in Hamlet: {known/total:.2%} ({known}/{total})')

Known n-grams in Sense and Sensibility: 15.10% (17925/118673)
Known n-grams in Hamlet: 3.06% (906/29603)


Let's add a method `generate` that takes the start of a sentence ("prompt") and a number of words to generate, then continues our prompt.

In [10]:
def generate(self, prompt, num_words=10):
    words = [w.lower() for w in prompt.split()]
    for i in range(num_words):
        prefix = tuple(words[-(self.n - 1):])
        if prefix not in self.counts:
            words.append('[END]')
            break
        next_word_dist = {}
        for ngram in self.counts:
            if len(ngram) == self.n and ngram[:-1] == prefix:
                next_word_dist[ngram] = self.counts[ngram] / self.counts[prefix]
        # print(prefix, next_word_dist)
        best_ngram, prob = max(next_word_dist.items(), key=lambda x: x[1])
        print(best_ngram, prob)
        words.append(best_ngram[-1])
    return ' '.join(words)

NGramLanguageModel.generate = generate

Play around with a few different prompts.

In [11]:
ngrams.generate("I went for a walk", num_words=20)

('a', 'walk', 'with') 0.3333333333333333
('walk', 'with', 'her,') 0.5
('with', 'her,', 'but') 0.21052631578947367
('her,', 'but', 'emma') 0.2
('but', 'emma', 'could') 0.5
('emma', 'could', 'not') 0.576271186440678
('could', 'not', 'be') 0.16605166051660517
('not', 'be', 'in') 0.03496503496503497
('be', 'in', 'a') 0.12121212121212122
('in', 'a', 'very') 0.056179775280898875
('a', 'very', 'good') 0.11413043478260869
('very', 'good', 'sort') 0.11904761904761904
('good', 'sort', 'of') 1.0
('sort', 'of', 'thing') 0.047619047619047616
('of', 'thing', 'that') 0.5
('thing', 'that', 'is') 0.13636363636363635
('that', 'is', 'quite') 0.1
('is', 'quite', 'a') 0.21052631578947367
('quite', 'a', 'different') 0.23809523809523808
('a', 'different', 'sort') 0.058823529411764705


'i went for a walk with her, but emma could not be in a very good sort of thing that is quite a different sort'